In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import statistics
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn import linear_model, ensemble, svm
from xgboost import XGBClassifier

In [ ]:
test = pd.read_csv('../input/airline-passenger-satisfaction/test.csv')
train = pd.read_csv('../input/airline-passenger-satisfaction/train.csv')

In [ ]:
train['satisfaction'] = train['satisfaction'].map({'neutral or dissatisfied' : 0, 'satisfied' : 1})
test['satisfaction'] = test['satisfaction'].map({'neutral or dissatisfied' : 0, 'satisfied' : 1})

In [ ]:
def one_hot_encode(db, feature):
    one_hot = pd.get_dummies(db[feature], prefix=feature)
    db = db.drop(feature, axis=1)
    return db.join(one_hot)

In [ ]:
def transform(x): 
    db = x.copy()
    db.pop('id')
    
    if 'satisfaction' in x:
        db.pop('satisfaction')
    db = one_hot_encode(db, 'Gender')
    db = one_hot_encode(db, 'Customer Type')
    db = one_hot_encode(db, 'Type of Travel')
    db = one_hot_encode(db, 'Class')
    return db

In [ ]:
X = train.copy()
X = transform(X)
X_test = test.copy()
X_test = transform(X_test)
y = train.satisfaction
y_test = test.satisfaction

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)
model = xgb.XGBClassifier(max_depth = 4, learning_rate = 0.1, n_estimators = 2000)
eval_set = [(X_val, y_val)]
model.fit(X_train, y_train, early_stopping_rounds=15, eval_set=eval_set)
y_pred = model.predict(X_val)
print("acc_score = ",accuracy_score(y_pred,y_val))

In [ ]:
y_test = model.predict(X_test)
df_answer = pd.DataFrame()
df_answer['id'] = test['id']
df_answer['satisfaction'] = y_test
df_answer.to_csv('answer.csv', index=False)